<a href="https://www.kaggle.com/code/arashnic/handling-imbalanced-resampling-the-right-way?scriptVersionId=177400499" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Introduction

This notebook is about how to do cross-validation once you have decided that oversampling is the right approach for imbalance problem.So  in this notebook i have focused on modeling , You can't fin feat eng , feat selection , hyper parameter optimization and so on.  If you want to play with the process yourself;you can complete the pipeline applying better algorithm(s) and feat eng as well. 

we will be going through the following steps:

- Getting a base model with random forest 
- Oversampling the wrong way: Do oversample, then oof cross-validate. Sounds fine, but results are overly optimistic.
- Oversampling the right way: Make SMOTE a part of our oof cross validation and compare recall and confusion matrix

also i used answer.npy , the target values of test data to evaluate unseen data for models.

Our goal will be to find a classifier with a good recall (i.e. we want our classifier to find as many positive cases as it can). So i didn't apply method to boost roc-auc for example. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file \\I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imbalanced-data-practice/sample_submission.csv
/kaggle/input/imbalanced-data-practice/aug_test.csv
/kaggle/input/imbalanced-data-practice/aug_train.csv
/kaggle/input/answer/answer.npy


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, GroupKFold, GridSearchCV, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsRegressor, KernelDensity, KDTree
from sklearn.metrics import *

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler, SMOTENC, SVMSMOTE
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import ClusterCentroids, TomekLinks, NearMiss, RandomUnderSampler, AllKNN, CondensedNearestNeighbour,\
                                        NeighbourhoodCleaningRule, OneSidedSelection, RepeatedEditedNearestNeighbours, InstanceHardnessThreshold



import sys, os
import random 

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from IPython.display import display


def set_seed(seed=4242):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

### Loading the data

In [3]:
train  = pd.read_csv('/kaggle/input/imbalanced-data-practice/aug_train.csv')
test  = pd.read_csv('/kaggle/input/imbalanced-data-practice/aug_test.csv')
answers = np.load('/kaggle/input/answer/answer.npy')
print(train.shape, test.shape)
display(train.head())
display(test.head())

(382154, 12) (78273, 11)


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,167647,Male,22,1,7.0,1,< 1 Year,No,2630.0,152.0,16,0
1,17163,Male,42,1,28.0,0,1-2 Year,Yes,43327.0,26.0,135,0
2,32023,Female,66,1,33.0,0,1-2 Year,Yes,35841.0,124.0,253,0
3,87447,Female,22,1,33.0,0,< 1 Year,No,27645.0,152.0,69,0
4,501933,Male,28,1,46.0,1,< 1 Year,No,29023.0,152.0,211,0


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,57782,Female,34,1,39.0,1,1-2 Year,No,38244.0,124.0,146
1,286811,Female,55,1,28.0,0,> 2 Years,Yes,37577.0,122.0,109
2,117823,Male,39,1,28.0,1,1-2 Year,No,24578.0,26.0,63
3,213992,Male,28,1,50.0,1,1-2 Year,No,40507.0,8.0,129
4,324756,Female,24,1,10.0,0,< 1 Year,Yes,36783.0,152.0,201


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382154 entries, 0 to 382153
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    382154 non-null  int64  
 1   Gender                382154 non-null  object 
 2   Age                   382154 non-null  int64  
 3   Driving_License       382154 non-null  int64  
 4   Region_Code           382154 non-null  float64
 5   Previously_Insured    382154 non-null  int64  
 6   Vehicle_Age           382154 non-null  object 
 7   Vehicle_Damage        382154 non-null  object 
 8   Annual_Premium        382154 non-null  float64
 9   Policy_Sales_Channel  382154 non-null  float64
 10  Vintage               382154 non-null  int64  
 11  Response              382154 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 35.0+ MB


In [5]:
train.describe(include=['O'])

,Gender,Vehicle_Age,Vehicle_Damage
count,382154,382154,382154
unique,2,3,2
top,Male,1-2 Year,No
freq,205603,200176,198501


### Categories

In [6]:
cats = [c for c in train.columns if train[c].dtypes =='object']
print('Categories', cats)

nums = [c for c in train.columns if c not in cats]
print('Numerics', nums)

Categories ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
Numerics ['id', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', 'Response']


In [7]:
for c in cats:
    le=LabelEncoder()
    le.fit(list(train[c].astype('str')) + list(test[c].astype('str')))
    train[c] = le.transform(list(train[c].astype(str))) 
    test[c] = le.transform(list(test[c].astype(str))) 
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,167647,1,22,1,7.0,1,1,0,2630.0,152.0,16,0
1,17163,1,42,1,28.0,0,0,1,43327.0,26.0,135,0
2,32023,0,66,1,33.0,0,0,1,35841.0,124.0,253,0
3,87447,0,22,1,33.0,0,1,0,27645.0,152.0,69,0
4,501933,1,28,1,46.0,1,1,0,29023.0,152.0,211,0


In [8]:
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,57782,0,34,1,39.0,1,0,0,38244.0,124.0,146
1,286811,0,55,1,28.0,0,2,1,37577.0,122.0,109
2,117823,1,39,1,28.0,1,0,0,24578.0,26.0,63
3,213992,1,28,1,50.0,1,0,0,40507.0,8.0,129
4,324756,0,24,1,10.0,0,1,1,36783.0,152.0,201


In [9]:
train.to_pickle('jc_train.pkl')
test.to_pickle('jc_test.pkl')